# Model selection and tuning
In this notebook we perform model selection and hyperparameter tuning.

We treat prediction of Russian as two-class classification problem and prediction of ITM as a regression problem and consider models appropriately.

For Russian, we consider gradient boosting models (`GradientBoostringClassifier` from *scikit-learn* library and `CatBoostClassifier` from *catboost* project), random forest (`RandomForestClassifier` from *scikit-learn*) and logistic regression (`LogisticRegression` from *scikit-learn*).

For ITM, we also consider gradient boosting (`GradientBoostringRegressor` and `CatBoostRegressor`) and random forest (`RandomForestRegressor`) as well as linear regression (`LinearRegression`). 

We avoid regularization in linear and logistic regression as we want unbaised estimations of the target variable.

Categorical variables (`sex`, `mother toungue` and `residence`) are processed with *one hot* encoding scheme for all models except *catboost* (these models has their own ways to deal with categorical variables).

First, we split dataset into two parts: with direct and indirect data. We perform optimization on each part separately.

As usual, each part of the dataset is splitted into train and test sample (70%/30%). Hyperparameter selection is performed using 10-fold cross validation on the train set. Then the best model is evaluated on the test set.

The optimization objective is negative log loss for Russian and $R^2$ for ITM because they provide unbaised estimate for the corresponding mean.

## imports

In [56]:
import joblib
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import json
from collections import Counter
from scipy import stats
from sklearn.ensemble import (
    GradientBoostingRegressor,
    GradientBoostingClassifier,
    RandomForestRegressor,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression, LinearRegression
from itertools import chain
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from indirect_utils import (
    logodds,
    stratified_permute,
    tologodds,
    trimmed,
    identity,
    read_data,
    residence_info,
    russian_to_target,
)
import random
import math
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score, log_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostClassifier, CatBoostRegressor
import pickle

pd.set_option("max_colwidth", 500)

%matplotlib inline

## Data loading and variable specification

In [13]:
data_ITM = read_data("ITM.csv")
data_russian = read_data("russian.csv").rename(columns={"русский": "russian"})

In [225]:
cat_features = [
    "sex",
    "mother tongue",  # L1, defined according to residence
    "residence",  # Village name
]

real_features = [
    "year_of_birth",
    "language population",
    "village population",
    "elevation",
]

## Model selection framework

In [44]:
def do_cv(estimator, param_grid, cat_features, real_features, target):
    """
    Makes cross-validation grid search for given estimator and param_grid.
    Split data into direct and indirect parts and train model separately on each part.
    Uses appropriate dataset and target variable for russian and ITM.
    target should be in ('russian', 'itm').
    returns dataframe with scores of best estimator.
    """
    assert target in ("russian", "itm")
    russian = target == "russian"
    if russian:
        scoring = lambda *args, **kwargs: -log_loss(*args, **kwargs)
        scoring_cv = "neg_log_loss"
        data = data_russian
    else:
        data = data_ITM
        scoring_cv = "r2"
        scoring = r2_score

    target = russian_to_target[russian]
    features = real_features + cat_features

    data_train, data_test = train_test_split(
        data[["type"] + features + [target]],
        test_size=0.3,
        stratify=data["type"],
        random_state=42,
    )
    
    if issubclass(estimator, CatBoostClassifier) or issubclass(
        estimator, CatBoostRegressor
    ):
        # Don't need OneHotEncoder() for CatBoost
        the_model = estimator()
        param_grid["cat_features"] = [cat_features]
    else:
        ct = ColumnTransformer(
        [
            ("catenc", OneHotEncoder(), cat_features),
            ("real", "passthrough", real_features),
        ])


        the_model = Pipeline([("column_transform", ct), ("estimator", estimator())])

    results = []
    for type in (0, 1):

        cv = GridSearchCV(the_model, param_grid, cv=10, n_jobs=-1, scoring=scoring_cv)
        predict = cv.predict_proba if russian else cv.predict

        data_train_mt, data_test_mt = (
            data_[lambda x: x["type"] == type] for data_ in (data_train, data_test)
        )

        cv.fit(data_train_mt.drop(target, axis=1), data_train_mt[target])

        train_score, test_score = (
            scoring(data_[target], predict(data_.drop(target, axis=1)))
            for data_ in (data_train_mt, data_test_mt)
        )

        results.append(
            [
                type,
                estimator.__name__,
                cv.cv_results_["mean_test_score"][cv.best_index_],
                cv.cv_results_["std_test_score"][cv.best_index_],
                cv.best_params_,
                train_score,
                test_score,
                cv,
            ]
        )

    return pd.DataFrame(
        results,
        columns=[
            "type",
            "estimator",
            "mean_cv_score",
            "std_cv_score",
            "cv_best_params",
            "train_score",
            "test_score",
            "cv",
        ],
    )

In [45]:
def compare_estimators(target, models, seed=None):
    if seed:
        np.random.seed(seed)
        
    return pd.concat(
        [
            do_cv(
                estimator,
                param_grid,
                cat_features=cat_features,
                real_features=real_features,
                target=target,
            )
            for estimator, param_grid in tqdm(models)
        ],
        axis=0,
        ignore_index=True,
    )

## Model selection
Actual calculation is done here. These cells will take a lot of time to execute.

In [46]:
itm_results = compare_estimators(
    target="itm",
    models=[
        (
            CatBoostRegressor,
            {
                "depth": [1, 2, 3, 4, 5, 6, 7, 8],
                "iterations": [20, 50, 100, 200, 500, 1000],
                "random_state": [42],
                "logging_level": ["Silent"],
            },
        ),
        (
            GradientBoostingRegressor,
            {
                "estimator__max_depth": [1, 2, 3, 4, 5, 6],
                "estimator__n_estimators": [
                    20,
                    50,
                    75,
                    100,
                    150,
                    200,
                    250,
                    300,
                    500,
                    1000,
                ],
                "estimator__random_state": [42],
            },
        ),
        (
            RandomForestRegressor,
            {
                "estimator__max_depth": [1, 2, 3, 4, 5, 6, 10, 20, 50, None],
                "estimator__n_estimators": [
                    20,
                    50,
                    75,
                    100,
                    150,
                    200,
                    250,
                    300,
                    1000,
                    2000,
                    5000,
                ],
                "estimator__random_state": [42],
            },
        ),
        (LinearRegression, {"estimator__fit_intercept": [True],}),
    ],
    seed=42,
)

/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [47]:
itm_results.drop(columns=['cv'])

,type,estimator,mean_cv_score,std_cv_score,cv_best_params,train_score,test_score
0,0,CatBoostRegressor,0.510373,0.075348,"{'cat_features': ['sex', 'mother tongue', 'residence'], 'depth': 5, 'iterations': 100, 'logging_level': 'Silent', 'random_state': 42}",0.605558,0.475098
1,1,CatBoostRegressor,0.470228,0.088613,"{'cat_features': ['sex', 'mother tongue', 'residence'], 'depth': 3, 'iterations': 500, 'logging_level': 'Silent', 'random_state': 42}",0.602121,0.427992
2,0,GradientBoostingRegressor,0.502545,0.071180,"{'estimator__max_depth': 2, 'estimator__n_estimators': 500}",0.597362,0.491880
3,1,GradientBoostingRegressor,0.475949,0.072724,"{'estimator__max_depth': 4, 'estimator__n_estimators': 75}",0.658705,0.412620
4,0,RandomForestRegressor,0.480104,0.059306,"{'estimator__max_depth': 6, 'estimator__n_estimators': 75, 'estimator__random_state': 42}",0.558588,0.450847
5,1,RandomForestRegressor,0.465799,0.082567,"{'estimator__max_depth': 10, 'estimator__n_estimators': 5000, 'estimator__random_state': 42}",0.746077,0.394373
6,0,LinearRegression,0.478275,0.068834,{'estimator__fit_intercept': True},0.505976,0.441011
7,1,LinearRegression,0.414478,0.093657,{'estimator__fit_intercept': True},0.481255,0.362570


In [49]:
russian_results = compare_estimators(
    target="russian",
    models=[
        (
            CatBoostClassifier,
            {
                "depth": [1, 2, 3, 4, 5, 6, 7, 8],
                "iterations": [20, 50, 100, 200, 500, 1000],
                "random_state": [42],
                "logging_level": ["Silent"],
            },
        ),
        (
            GradientBoostingClassifier,
            {
                "estimator__max_depth": [1, 2, 3, 4, 5, 6],
                "estimator__n_estimators": [
                    20,
                    50,
                    75,
                    100,
                    150,
                    200,
                    250,
                    300,
                    500,
                    1000,
                ],
                "estimator__random_state": [42],


            },
        ),
        (
            RandomForestClassifier,
            {
                "estimator__max_depth": [1, 2, 3, 4, 5, 6, 10, 20, 50, None],
                "estimator__n_estimators": [
                    20,
                    50,
                    75,
                    100,
                    150,
                    200,
                    250,
                    300,
                    1000,
                    2000,
                    5000,
                ],
                "estimator__criterion": ["gini", "entropy"],
                "estimator__random_state": [42],
            },
        ),
        (LogisticRegression, {"estimator__penalty": ["none"],}),
    ],
    seed=42,
)

/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/gleb/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
 

In [54]:
russian_results.drop(columns=['cv'])

,type,estimator,mean_cv_score,std_cv_score,cv_best_params,train_score,test_score
0,0,CatBoostClassifier,-0.375182,0.027534,"{'cat_features': ['sex', 'mother tongue', 'residence'], 'depth': 4, 'iterations': 200, 'logging_level': 'Silent', 'random_state': 42}",-0.340403,-0.389767
1,1,CatBoostClassifier,-0.232651,0.037873,"{'cat_features': ['sex', 'mother tongue', 'residence'], 'depth': 5, 'iterations': 20, 'logging_level': 'Silent', 'random_state': 42}",-0.205049,-0.260945
2,0,GradientBoostingClassifier,-0.374709,0.025637,"{'estimator__max_depth': 1, 'estimator__n_estimators': 300}",-0.353653,-0.387366
3,1,GradientBoostingClassifier,-0.233755,0.044909,"{'estimator__max_depth': 1, 'estimator__n_estimators': 200}",-0.202351,-0.249073
4,0,RandomForestClassifier,-0.385647,0.017245,"{'estimator__criterion': 'entropy', 'estimator__max_depth': 10, 'estimator__n_estimators': 2000, 'estimator__random_state': 42}",-0.306238,-0.391133
5,1,RandomForestClassifier,-0.241468,0.046484,"{'estimator__criterion': 'entropy', 'estimator__max_depth': 10, 'estimator__n_estimators': 50, 'estimator__random_state': 42}",-0.145593,-0.263121
6,0,LogisticRegression,-0.480567,0.033679,{'estimator__penalty': 'none'},-0.526693,-0.521939
7,1,LogisticRegression,-0.336939,0.017054,{'estimator__penalty': 'none'},-0.335783,-0.377689


In [223]:
itm_results.pivot(index="estimator", columns="type", values="test_score").sort_values(
    0, ascending=False
)

type,0,1
estimator,,
GradientBoostingRegressor,0.491880,0.412620
CatBoostRegressor,0.475098,0.427992
RandomForestRegressor,0.450847,0.394373
LinearRegression,0.441011,0.362570


In [224]:
russian_results.pivot(
    index="estimator", columns="type", values="test_score"
).sort_values(0, ascending=False)

type,0,1
estimator,,
GradientBoostingClassifier,-0.387366,-0.249073
CatBoostClassifier,-0.389767,-0.260945
RandomForestClassifier,-0.391133,-0.263121
LogisticRegression,-0.521939,-0.377689


As we can see, gradient boosting models (*scikit-learn*'s `GradientBoosting*` and `CatBoost*`) outperform the other models in both settings, though advantage over `RandomForest` is rather small (but seem to be consistent according to our experiments). `CatBoostRegressor` appear to be marginally better than `GradientBoostingRegressor` in ITM prediction with direct data (though this is unstable and varies with random seed), but we will stick with *scikit-learn*'s `GradientBoosting*` models because they are faster.

We will use hyperparameters found during tuning in the subsequent experiments.

In [57]:
with open("russian_cv_model_select.pickle", "wb") as f:
    pickle.dump(russian_results, f)
    
with open("itm_cv_model_select.pickle", "wb") as f:
    pickle.dump(itm_results, f)